# Trend Screener

In [ ]:
pip install "yfinance[nospam]" stocktrends matplotlib pandas numpy requests_cache

In [ ]:
import yfinance as yf
import pandas as pd
import stocktrends as st
import numpy as np
import requests_cache
from concurrent.futures import ThreadPoolExecutor, as_completed

session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'Mozilla/5.0 (compatible; MSIE 7.0; Windows; U; Windows NT 10.4; Win64; x64; en-US Trident/4.0)'

results = []

tickers_txt_path = 'us_stock.txt'

try:
    with open(tickers_txt_path, 'r') as file:
        tickers = [line.strip() for line in file if line.strip()]
except Exception as e:
    print(f"Error reading tickers from text file: {e}")
    tickers = []

start = '2023-03-26'
end = '2024-07-26'

data = yf.download(tickers, start = start, end = end, session=session)

print(data)

In [ ]:
def ehlers_instantaneous_trendline(source, alpha=0.07):
    itrend = np.zeros_like(source)
    trigger = np.zeros_like(source)

    for i in range(len(source)):
        if i < 7:
            itrend[i] = (source.iloc[i] + (2 * source.iloc[i - 1]) + source.iloc[i - 2]) / 4
        else:
            itrend[i] = (
                ((alpha - (alpha ** 2 / 4)) * source.iloc[i]) +
                (0.5 * alpha ** 2 * source.iloc[i - 1]) -
                ((alpha - (0.75 * alpha ** 2)) * source.iloc[i - 2]) +
                (2 * (1 - alpha) * itrend[i - 1]) -
                ((1 - alpha) ** 2 * itrend[i - 2])
            )

        if i >= 2:
            trigger[i] = (2 * itrend[i]) - itrend[i - 2]

    return itrend

def calculate_trend(df):
    df['52W_High'] = df['High'][-317:].rolling(window=252).max()
    df['52W_High_Ratio'] = df['High'][-1:] / df['52W_High'][-1:]
    df['52W_ROC'] = (df['High'][-317:] - df['High'].shift(252)) / df['High'].shift(252)
    
    df['52W_Low'] = df['Low'][-252:].rolling(window=252).min()
    df['52W_Low_Ratio'] = df['Low'][-1:] / df['52W_Low'][-1:]


    df['26W_High'] = df['High'].rolling(window=126).max()
    df['26W_High_Ratio'] = df['High'] / df['26W_High']
    df['26W_ROC'] = (df['High'][-146:] - df['High'].shift(126)) / df['High'].shift(252)

    df['50MA'] = df['Adj Close'][-51:].rolling(window=50).mean()
    df['150MA'] = df['Adj Close'][-151:].rolling(window=150).mean()
    df['200MA'] = df['Adj Close'][-282:].rolling(window=200).mean()

    df['MA_Uptrend'] = (
        (df['Adj Close'][-2:] > df['50MA'][-2:]) & 
        (df['50MA'][-2:] > df['150MA'][-2:]) &
        (df['150MA'][-2:] > df['200MA'][-2:]) &
        (df['50MA'][-2:].diff() > 0) &
        (df['150MA'][-2:].diff() > 0) &
        (df['200MA'][-2:].diff() > 0)
    )

    df['200MA_Uptrend'] = df['200MA'][-81:].diff() > 0
    df['200MA_1M_Uptrend'] = df['200MA_Uptrend'][-20:].rolling(window=20).mean() * 100
    df['200MA_4M_Uptrend'] = df['200MA_Uptrend'][-80:].rolling(window=80).mean() * 100

    itrend = ehlers_instantaneous_trendline(df['Adj Close'])
    df['ITrend'] = itrend
    df['ITrend_Uptrend'] = df['ITrend'][-2:].diff() > 0

    df['Stop_Loss'] = 1 - df['50MA'][-1:] / df['Adj Close'][-1:]
    df['g'] = df['ITrend'][-1:] / df['ITrend'].shift(1) - 1
    df['Risk_Reward'] = df['g'][-1:] / df['Stop_Loss'][-1:]

    return df

def process_trend(ticker):
    try:
        print(f"process trend {ticker}")
        ticker_df = pd.DataFrame(data['Adj Close'][ticker])
        ticker_df['High'] = data['High'][ticker]
        ticker_df['Low'] = data['Low'][ticker]
        ticker_df = ticker_df.rename(columns={ticker: 'Adj Close'})
        ticker_df = calculate_trend(ticker_df)

        return ticker, ticker_df
    except Exception as e:
        print(f"Error calculating trend for {ticker}: {e}")
        return ticker, None

results = {}

with ThreadPoolExecutor(max_workers=20) as executor:
    futures = {executor.submit(process_trend, ticker): ticker for ticker in tickers}
    for future in as_completed(futures):
        ticker, result = future.result()
        if result is not None:
            results[ticker] = result


roc52w_data = pd.DataFrame({ticker: df['52W_ROC'] for ticker, df in results.items()})
roc52w_ranks = roc52w_data.rank(axis=1)
roc52w_ranks = roc52w_ranks.div(len(roc52w_ranks.columns))

roc26w_data = pd.DataFrame({ticker: df['26W_ROC'] for ticker, df in results.items()})
roc26w_ranks = roc26w_data.rank(axis=1)
roc26w_ranks = roc26w_ranks.div(len(roc26w_ranks.columns))

def process_rs(ticker):
    try:
        print(f"process rs {ticker}")
        results[ticker]['RS52W'] = roc52w_ranks[ticker]
        results[ticker]['RS52W_13W'] = results[ticker]['RS52W'].rolling(window=65).mean()

        results[ticker]['RS26W'] = roc26w_ranks[ticker]
        results[ticker]['RS26W_4W'] = results[ticker]['RS26W'].rolling(window=20).mean()
    except Exception as e:
        print(f"Error adding RS for {ticker}: {e}")

with ThreadPoolExecutor(max_workers=20) as executor:
    futures = {executor.submit(process_rs, ticker): ticker for ticker in tickers}
    for future in as_completed(futures):
        future.result()

In [122]:
summary_data = {}
for ticker, df in results.items():
    latest_data = df.iloc[-1]
    uptrend = latest_data['52W_High_Ratio'] >= 0.75 and \
        latest_data['52W_Low_Ratio'] >= 1.25 and \
        latest_data['MA_Uptrend'] and \
        latest_data['ITrend_Uptrend'] and \
        latest_data['g'] > 0.0 and \
        latest_data['RS26W'] > 0.7 and \
        latest_data['RS52W'] > 0.7 and \
        latest_data['Stop_Loss'] > 0.02

    if uptrend:
      summary_data[ticker] = {
          '52W_High_Ratio': round(latest_data['52W_High_Ratio'], 2),
          '52W_ROC': round(latest_data['52W_ROC'], 2),
          '26W_High_Ratio': round(latest_data['26W_High_Ratio'], 2),
          '26W_ROC': round(latest_data['26W_ROC'], 2),
          'RS52W': round(latest_data['RS52W'], 2),
          'RS52W_13W': round(latest_data['RS52W_13W'], 2),
          'RS26W': round(latest_data['RS26W'], 2),
          'RS26W_4W': round(latest_data['RS26W_4W'], 2),
          'Stop_Loss': round(latest_data['Stop_Loss'], 2),
          'g': round(latest_data['g'] * 100, 4),
          'Risk_Reward': round(latest_data['Risk_Reward'], 2),
          'MA_Uptrend': latest_data['MA_Uptrend'],
          'ITrend_Uptrend': latest_data['ITrend_Uptrend'],
          'Uptrend': uptrend
      }

summary_df = pd.DataFrame(summary_data).T
summary_df = summary_df.sort_values(by=['Uptrend', 'RS26W'], ascending=[False, False])

summary_df.to_csv('filtered_stocks.csv')

print(summary_df)

     52W_High_Ratio 52W_ROC 26W_High_Ratio 26W_ROC RS52W RS52W_13W RS26W  \
ADMA           0.98    2.64           0.98    2.23  0.87      0.85  0.87   
APEI           0.98     3.3           0.98    1.68  0.87      0.86  0.87   
ASTS            1.0    3.27            1.0    3.44  0.87      0.58  0.87   
BNTC           0.87    1.54           0.87    1.74  0.86      0.83  0.87   
CRBP           0.92    7.32           0.92    7.11  0.87      0.87  0.87   
...             ...     ...            ...     ...   ...       ...   ...   
MC             0.99    0.29           0.99    0.22   0.7      0.71   0.7   
MORN            1.0    0.52            1.0    0.22  0.78      0.77   0.7   
MUFG           0.97    0.47           0.97    0.22  0.77      0.77   0.7   
R              0.99    0.47           0.99    0.22  0.77      0.77   0.7   
TASK           0.98     0.4           0.98    0.21  0.75       0.6   0.7   

     RS26W_4W Stop_Loss       g Risk_Reward MA_Uptrend ITrend_Uptrend Uptrend  
ADMA   